<a href="https://colab.research.google.com/github/Ankur3107/GitHub-Bugs-Prediction-Challenge/blob/main/model-template/MachineHack_roberta_base.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers

     |████████████████████████████████| 1.1MB 3.4MB/s 
     |████████████████████████████████| 1.1MB 17.5MB/s 
     |████████████████████████████████| 3.0MB 27.2MB/s 
     |████████████████████████████████| 890kB 26.1MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=2e9363558ea8887396422c1b90e23e7cced22406d1ce44ff82dd836a65f0bca3
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
!wget https://machinehack-be.s3.amazonaws.com/predict_github_issues_embold_sponsored_hackathon/Embold_Participant%27s_Dataset.zip
!unzip ./Embold_Participant\'s_Dataset.zip -d Dataset

--2020-10-15 12:36:30--  https://machinehack-be.s3.amazonaws.com/predict_github_issues_embold_sponsored_hackathon/Embold_Participant%27s_Dataset.zip
Resolving machinehack-be.s3.amazonaws.com (machinehack-be.s3.amazonaws.com)... 52.219.62.68
Connecting to machinehack-be.s3.amazonaws.com (machinehack-be.s3.amazonaws.com)|52.219.62.68|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 102320961 (98M) [application/octet-stream]
Saving to: ‘Embold_Participant's_Dataset.zip’

Embold_Participant' 100%[===================>]  97.58M  12.3MB/s    in 9.5s    

2020-10-15 12:36:41 (10.3 MB/s) - ‘Embold_Participant's_Dataset.zip’ saved [102320961/102320961]

Archive:  ./Embold_Participant's_Dataset.zip
   creating: Dataset/Embold_Participant's_Dataset/
  inflating: Dataset/Embold_Participant's_Dataset/sample submission.csv  
  inflating: Dataset/__MACOSX/Embold_Participant's_Dataset/._sample submission.csv  
  inflating: Dataset/Embold_Participant's_Dataset/embold_train_extra

In [ ]:
cd "Dataset/Embold_Participant's_Dataset/"

/content/Dataset/Embold_Participant's_Dataset


In [ ]:
#import pandas as pd
import numpy as np
import pandas as pd

In [ ]:
train_df = pd.read_json("embold_train.json").reset_index(drop=True)
test_df = pd.read_json("embold_test.json").reset_index(drop=True)
train_ex_df = pd.read_json("embold_train_extra.json")

In [ ]:
test_label_df = pd.read_csv('/content/ensemble_v8_1.csv')
test_df['label'] = test_label_df['label']

In [ ]:
train_data = train_df.append(train_ex_df)
test_df['text'] = test_df['title']+' '+test_df['body']
train_data['text'] = train_data['title']+' '+train_data['body']

In [ ]:
from dataclasses import dataclass
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from transformers import *
from transformers import pipeline
from pprint import pprint
from tensorflow import keras

In [ ]:
@dataclass
class Config:
    MAX_LEN = 320
    BATCH_SIZE = 16  # per TPU core
    TOTAL_STEPS = 2000  # thats approx 4 epochs
    EVALUATE_EVERY = 200
    LR = 1e-5
    PRETRAINED_MODEL = "roberta-large"  # huggingface bert model


transformer_flags = Config()


def connect_to_TPU():
    """Detect hardware, return appropriate distribution strategy"""
    try:
        # TPU detection. No parameters necessary if TPU_NAME environment variable is
        # set: this is always the case on Kaggle.
        tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
        print("Running on TPU ", tpu.master())
    except ValueError:
        tpu = None

    if tpu:
        tf.config.experimental_connect_to_cluster(tpu)
        tf.tpu.experimental.initialize_tpu_system(tpu)
        strategy = tf.distribute.experimental.TPUStrategy(tpu)
    else:
        # Default distribution strategy in Tensorflow. Works on CPU and single GPU.
        strategy = tf.distribute.get_strategy()

    global_batch_size = transformer_flags.BATCH_SIZE * strategy.num_replicas_in_sync

    return tpu, strategy, global_batch_size


tpu, strategy, global_batch_size = connect_to_TPU()
print("REPLICAS: ", strategy.num_replicas_in_sync)

INFO:absl:Entering into master device scope: /job:worker/replica:0/task:0/device:CPU:0


Running on TPU  grpc://10.17.198.186:8470
INFO:tensorflow:Initializing the TPU system: grpc://10.17.198.186:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.17.198.186:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


REPLICAS:  8


In [ ]:
def regular_encode(texts, tokenizer, maxlen=512):
    enc_di = tokenizer.batch_encode_plus(
        texts,
        return_attention_mask=False,
        return_token_type_ids=False,
        pad_to_max_length=True,
        max_length=maxlen,
        truncation=True,
    )

    return np.array(enc_di["input_ids"])

from transformers import *
#tokenizer = AutoTokenizer.from_pretrained(transformer_flags.PRETRAINED_MODEL)
#tokenizer = BertTokenizerFast.from_pretrained(transformer_flags.PRETRAINED_MODEL)
#tokenizer = AlbertTokenizer.from_pretrained(transformer_flags.PRETRAINED_MODEL, use_fast=True)
tokenizer = RobertaTokenizerFast.from_pretrained(transformer_flags.PRETRAINED_MODEL)
#tokenizer = ElectraTokenizerFast.from_pretrained(transformer_flags.PRETRAINED_MODEL)

In [ ]:
def build_model():
  input_layer = tf.keras.layers.Input((transformer_flags.MAX_LEN,), dtype=tf.int32)
  bert_model = TFAutoModel.from_pretrained(transformer_flags.PRETRAINED_MODEL, from_pt=True)
  #bert_model = TFAutoModel.from_pretrained('/content/github-albert-xlarge-v2/')
  #bert_model = TFAutoModel.from_pretrained('../../github-roberta-large/')
  output_layer = bert_model(input_layer) 
  pooled_output = tf.keras.layers.GlobalAveragePooling1D()(output_layer[0])
  output = tf.keras.layers.Dense(3, activation='softmax')(pooled_output)
  classifier_model = tf.keras.Model(input_layer, output)

  optimizer = tf.keras.optimizers.Adam(learning_rate=transformer_flags.LR)
  classifier_model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
  classifier_model.summary()
    
  return classifier_model

In [ ]:
all_train = train_data[0:200000]
all_train = all_train.append(train_data[230000:])

In [ ]:
question_df = all_train[all_train.label==2]
bug_issue = all_train[all_train.label!=2]
bug_issue = bug_issue.sample(50000)
train = question_df.append(bug_issue)
train = train.append(test_df)
train = train.sample(len(train))

In [ ]:
train.label.value_counts()

2    41662
1    39361
0    38302
Name: label, dtype: int64

In [ ]:
validate = train_data[200000:230000]
validate.label.value_counts()

1    13846
0    13278
2     2876
Name: label, dtype: int64

In [ ]:
with strategy.scope():
    classifier_model = build_model()

In [ ]:
# data preparation
X_data = regular_encode(train.text.values.tolist(), tokenizer, maxlen=transformer_flags.MAX_LEN)
y_train = train.label.values
print(X_data.shape, y_train.shape)

In [ ]:
# data preparation
X_val = regular_encode(validate.text.values.tolist(), tokenizer, maxlen=transformer_flags.MAX_LEN)
y_val = validate.label.values
print(X_val.shape, y_val.shape)

In [ ]:
X_test = regular_encode(test_df.text.values.tolist(), tokenizer, maxlen=transformer_flags.MAX_LEN)

In [ ]:
# 0:200000+30000 datapoint
classifier_model.fit(X_data, y_train, epochs=1,batch_size=64, validation_data=(X_val, y_val))

Epoch 1/2
Instructions for updating:
Use `tf.data.Iterator.get_next_as_optional()` instead.


Instructions for updating:
Use `tf.data.Iterator.get_next_as_optional()` instead.


   2/3427 [..............................] - ETA: 17:51:24 - loss: 1.1823 - accuracy: 0.2969WARNING:tensorflow:Callbacks method `on_train_batch_end` is slow compared to the batch time (batch time: 0.0115s vs `on_train_batch_end` time: 0.1867s). Check your callbacks.


3427/3427 [==============================] - ETA: 0s - loss: 0.5101 - accuracy: 0.8049WARNING:tensorflow:Callbacks method `on_test_batch_end` is slow compared to the batch time (batch time: 0.0039s vs `on_test_batch_end` time: 0.0545s). Check your callbacks.


3427/3427 [==============================] - 763s 223ms/step - loss: 0.5101 - accuracy: 0.8049 - val_loss: 0.4532 - val_accuracy: 0.8303
Epoch 2/2
3427/3427 [==============================] - 710s 207ms/step - loss: 0.4480 - accuracy: 0.8330 - val_loss: 0.4552 - val_accuracy: 0.8275


In [ ]:
# 0:200000+30000 datapoint
classifier_model.fit(X_data, y_train, epochs=1,batch_size=64, validation_data=(X_val, y_val))

   2/3427 [..............................] - ETA: 11:01 - loss: 0.4318 - accuracy: 0.8828WARNING:tensorflow:Callbacks method `on_train_batch_end` is slow compared to the batch time (batch time: 0.0090s vs `on_train_batch_end` time: 0.1855s). Check your callbacks.


3427/3427 [==============================] - ETA: 0s - loss: 0.4339 - accuracy: 0.8391WARNING:tensorflow:Callbacks method `on_test_batch_end` is slow compared to the batch time (batch time: 0.0044s vs `on_test_batch_end` time: 0.0520s). Check your callbacks.


3427/3427 [==============================] - 713s 208ms/step - loss: 0.4339 - accuracy: 0.8391 - val_loss: 0.4428 - val_accuracy: 0.8331


In [ ]:
# 0:200000+30000 datapoint
classifier_model.fit(X_data, y_train, epochs=1,batch_size=64, validation_data=(X_val, y_val))

   2/2646 [..............................] - ETA: 8:59 - loss: 0.5107 - accuracy: 0.7812WARNING:tensorflow:Callbacks method `on_train_batch_end` is slow compared to the batch time (batch time: 0.0108s vs `on_train_batch_end` time: 0.1876s). Check your callbacks.


2646/2646 [==============================] - ETA: 0s - loss: 0.4202 - accuracy: 0.8443WARNING:tensorflow:Callbacks method `on_test_batch_end` is slow compared to the batch time (batch time: 0.0045s vs `on_test_batch_end` time: 0.0534s). Check your callbacks.


2646/2646 [==============================] - 593s 224ms/step - loss: 0.4202 - accuracy: 0.8443 - val_loss: 0.4492 - val_accuracy: 0.8311


In [ ]:
# 0:200000+30000 datapoint
classifier_model.fit(X_data, y_train, epochs=1,batch_size=64, validation_data=(X_val, y_val))

   2/1865 [..............................] - ETA: 5:58 - loss: 0.4215 - accuracy: 0.8516WARNING:tensorflow:Callbacks method `on_train_batch_end` is slow compared to the batch time (batch time: 0.0105s vs `on_train_batch_end` time: 0.1836s). Check your callbacks.


1865/1865 [==============================] - ETA: 0s - loss: 0.3891 - accuracy: 0.8549WARNING:tensorflow:Callbacks method `on_test_batch_end` is slow compared to the batch time (batch time: 0.0042s vs `on_test_batch_end` time: 0.0514s). Check your callbacks.


1865/1865 [==============================] - 434s 233ms/step - loss: 0.3891 - accuracy: 0.8549 - val_loss: 0.4763 - val_accuracy: 0.8216


In [ ]:
# 0:200000+30000 datapoint
classifier_model.fit(X_data, y_train, epochs=1,batch_size=64, validation_data=(X_val, y_val))

   2/1552 [..............................] - ETA: 5:30 - loss: 0.3634 - accuracy: 0.8594WARNING:tensorflow:Callbacks method `on_train_batch_end` is slow compared to the batch time (batch time: 0.0115s vs `on_train_batch_end` time: 0.1851s). Check your callbacks.


1552/1552 [==============================] - ETA: 0s - loss: 0.3402 - accuracy: 0.8749WARNING:tensorflow:Callbacks method `on_test_batch_end` is slow compared to the batch time (batch time: 0.0043s vs `on_test_batch_end` time: 0.0518s). Check your callbacks.


1552/1552 [==============================] - 338s 218ms/step - loss: 0.3402 - accuracy: 0.8749 - val_loss: 0.5947 - val_accuracy: 0.7892


In [ ]:
y_val_pred = classifier_model.predict(X_val, batch_size=global_batch_size, verbose=1)
y_test_pred = classifier_model.predict(X_test, batch_size=global_batch_size, verbose=1)

  2/235 [..............................] - ETA: 10sWARNING:tensorflow:Callbacks method `on_predict_batch_end` is slow compared to the batch time (batch time: 0.0039s vs `on_predict_batch_end` time: 0.0872s). Check your callbacks.


  2/235 [..............................] - ETA: 10sWARNING:tensorflow:Callbacks method `on_predict_batch_end` is slow compared to the batch time (batch time: 0.0039s vs `on_predict_batch_end` time: 0.0867s). Check your callbacks.


235/235 [==============================] - 21s 91ms/step


In [ ]:
eval_df = pd.DataFrame({'X':y_val_pred.tolist(), 'y':y_val.tolist()})

In [ ]:
eval_df.to_csv('textattack-roberta-base-MNLI-maxlen320-PL-6epochs-90K87K40K-eval-prob.csv', index=False)

In [ ]:
submission = pd.read_csv('sample submission.csv')
submission['label'] = y_test_pred.tolist()
file_name = 'textattack-roberta-base-MNLI-maxlen320-PL-6epochs-90K87K40K-test-prob.csv'
submission.to_csv(file_name, index=False)